## ВКР

Автоматизация подбора персонала

Копчев Владислав, БПМИ197

### Библиотеки

selenium поскольку нужен js

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
# !pip install transliterate
from transliterate import translit

### Скрейпинг страницы со списком резюме

hh.ru не принимает названия с заглавными буквами:

In [78]:
def normalize_translit(ru_query):
    text = translit(ru_query, language_code='ru', reversed=True)
    text = text.replace(' ', '-').replace('yj', 'yy').replace('yy-', 'yy_').replace('es-', 'es_').replace('k-pr', 'k_pr')  # эвристическое правило, не знаю, почему у hh так...
    return text

In [82]:
driver = webdriver.Chrome('./chromedriver')

def scrape(query, driver):
    pages_count = 10  # соскрейпить это число иначе ошибки!
    for n in range(pages_count):  # проблема!!
        link = 'https://hh.ru/resumes/{}?items_on_page=100&page={}'.format(query, n)  # 100 per page
        driver.get(link)

        resumes_list = driver.find_element(by=By.XPATH, value='//div[@class="resume-search-item__header"]')
        # resumes_list.click()  # стоп а зачем это...
        resumes = resumes_list.find_element(by=By.XPATH, value='//div[@data-qa="resume-serp__results-search"]').find_elements(by=By.XPATH, value='//a[@class="serp-item__title"]')

        resumes_parsed = []
        for x in resumes:
            resumes_parsed.append((x.text, x.get_attribute("href")))

        if n == 0:
            df = pd.DataFrame(resumes_parsed, columns=['Вакансия', 'Ссылка'])
            # print(len(resumes_parsed))
        else:
            # print(len(resumes_parsed))
            df2 = pd.DataFrame(resumes_parsed, columns=['Вакансия', 'Ссылка'])  # , index=list(range(0 + 100 * n, 100 + 100 * n)) n00, ..., n99n = 0, ..., 49
            df = pd.concat([df, df2])
    
    return df

<ipython-input-82-c83a548329b0>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [83]:
analysts_list = ['аналитик bi',
                'системный аналитик',
                'бизнес аналитик',
                'аналитик продаж',
                'финансовый аналитик',
                'аналитик данных',
                'data analyst']
analysts_map = dict()

for analyst in analysts_list:
    analysts_map[analyst] = scrape(normalize_translit(analyst), driver)
    analysts_map[analyst].index = range(analysts_map[analyst].shape[0])

Конкатеннация результатов:

In [85]:
df1 = pd.DataFrame()

for analyst in analysts_list:
    df2 = analysts_map[analyst]
    df1 = pd.concat([df1, df2])
    
df1.index = range(df1.shape[0])
df1.to_csv('resumes_all.csv')

### Скрейпинг страницы с конкретным резюме

In [126]:
def normalize(x):
    if x:
        x = x[0].text
    else:
        x = '---'  # так будет лучше чем просто ' '?
    return x

In [127]:
def scrape(link, driver):
    driver.get(link)
    comandirovka = driver.find_elements(by=By.XPATH, value='//div[@class="bloko-translate-guard"]')
    comandirovka = normalize(comandirovka)  # норм?
    
    opyt = driver.find_elements(by=By.XPATH, value='//span[@class="resume-block__title-text resume-block__title-text_sub"]')
    opyt = normalize(opyt)

    about = driver.find_elements(by=By.XPATH, value='//div[@class="resume-block-container"][@data-qa="resume-block-skills-content"]')
    about = normalize(about)
        
    educ = driver.find_elements(by=By.XPATH, value='//div[@class="resume-block"][@data-qa="resume-block-education"]')
    educ = normalize(educ)
        
    inter = driver.find_elements(by=By.XPATH, value='//div[@class="resume-block-item-gap"]')
    inter = normalize(inter)
        
    inter2 = driver.find_elements(by=By.XPATH, value='//div[@class="key-skills-row"][@data-qa="tags-key-skills"]')
    inter2 = normalize(inter2)
    
    # Образование 
    
    educ_val = '//div[@data-qa="resume-block-education"][@class="resume-block"]//div[@data-qa="resume-block-education-name"]'
    educ = driver.find_elements(by=By.XPATH, value=educ_val)
    stepen_val = '//div[@data-qa="resume-block-education"]//span[@class="resume-block__title-text resume-block__title-text_sub"]'
    stepen = driver.find_elements(by=By.XPATH, value=stepen_val)
    title_val = '//span[@class="resume-block__title-text"][@data-qa="resume-block-title-position"]'
    title = driver.find_elements(by=By.XPATH, value=title_val)
    if title:
        title = educ[0].text
    else:
        title = ''
    
    educ1 = ''
    educ2 = ''
    educ3 = ''
    
    if educ:
        educ1 = educ[0].text
    if len(educ) > 1:
        educ2 = educ[1].text
    if stepen:
        educ3 = stepen[0].text
    vuzes_count = len(educ)
    
    # Проблема: иногда уровень образования могут не писать
        
    return (link, title, comandirovka, opyt, about, educ, inter, inter2, educ1, educ2, educ3, vuzes_count)

In [128]:
df_links = list(pd.read_csv('resumes_all.csv')['Ссылка'])[:100]

resume_list = []
driver = webdriver.Chrome('./chromedriver')
for link in df_links:
    # print(link)
    data = scrape(link, driver)
    resume_list.append(data)

df = pd.DataFrame(resume_list, columns=['Ссылка',
                                        'Название',
                                        'Коммандировка', 
                                        'Опыт',
                                        'О себе',
                                        'Образование',
                                        'Интересы',
                                        'Навыки',
                                        'Образование-1',
                                        'Образование-2',
                                        'Уровень образование',
                                        'Кол-во образований'])
df.to_csv('resumes_features.csv')
df.head(2)

<ipython-input-128-1ea1f9da7e59>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


MOSCOW STATE UNIVERSITY OF MECHANICAL ENGINEERING (MAMI), Moscow

Санкт-Петербургский государственный политехнический университет, Санкт-Петербург

Самарский государственный аэрокосмический университет им. академика С.П. Королева (национальный исследовательский университет), Самара

Дальневосточный государственный технический университет, Владивосток

Челябинский государственный университет, Челябинск

Национальный исследовательский университет "Высшая школа экономики", Москва

Московский гуманитарно-экономический институт, Москва

Graduate School of Management

Московский государственный технический университет им. Н.Э. Баумана, Москва

Российский экономический университет им. Г.В. Плеханова, Москва

Финансовый университет при Правительстве Российской Федерации, Москва

Российский технологический университет, Москва

Московский Инженерно-Физический Институт (государственный университет)

Российский государственный социальный университет, Москва

Национальный исследовательский универси

,Ссылка,Название,Коммандировка,Опыт,О себе,Образование,Интересы,Навыки,Образование-1,Образование-2,Уровень образование,Кол-во образований
0,https://hh.ru/resume/1aa91e410000fd3cda0039ed1...,MOSCOW STATE UNIVERSITY OF MECHANICAL ENGINEER...,"Moscow, willing to relocate, prepared for occa...",Work experience 7 years 4 months,"Responsible, communicable, quick study and det...",[<selenium.webdriver.remote.webelement.WebElem...,"Specializations:\nSales manager, account manag...",Поддержка продаж / владение ПО для поддержки п...,MOSCOW STATE UNIVERSITY OF MECHANICAL ENGINEER...,,Higher education,1
1,https://hh.ru/resume/bebf6a630008069d8c0039ed1...,Санкт-Петербургский государственный политехнич...,"Санкт-Петербург, м. Гражданский проспект, гото...",Опыт работы 7 лет 2 месяца,Имею экономическое образование. Продвинутый по...,[<selenium.webdriver.remote.webelement.WebElem...,Специализации:\nАналитик\nЗанятость: полная за...,"Работа с компьютерными программами, программы ...",Санкт-Петербургский государственный политехнич...,,Высшее образование (Бакалавр),1


**Справочник по образованию:**

Найти подобные списки для ТОП-10 вузов и проанализировать:
https://propostuplenie.ru/article/abbreviatury-i-prozvishcha-populyarnyh-vuzov/?ysclid=la9p7r8feo708199013

Топ вузов: https://vuzoteka.ru/вузы?ysclid=la9p8mfldl600081011